In [ ]:
# This process involves three main stages:
# 2.Fetching data from a weather API using Python.
# 3.Structuring the data for database storage.
# 4.Inserting the data into a PostgreSQL database.

#In Terminal: python -m pip install psycopg2-binary


In [ ]:
#Initialize the database
#initdb /usr/local/var/postgres

In [10]:
import requests

In [11]:
url = "https://api.weather.gov/points/38.9012,-77.2653"
response = requests.get(url)
data = response.json()

print(data.keys()) #to explore structure

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])


In [12]:
forecast_hourly_url = data["properties"]["forecastHourly"]
print(forecast_hourly_url)

https://api.weather.gov/gridpoints/LWX/89,70/forecast/hourly


In [13]:
# Fetch hourly forecast data
hourly_response = requests.get(forecast_hourly_url)
hourly_data = hourly_response.json()

periods = hourly_data["properties"]["periods"]
print(periods[0])

{'number': 1, 'name': '', 'startTime': '2025-11-12T22:00:00-05:00', 'endTime': '2025-11-12T23:00:00-05:00', 'isDaytime': False, 'temperature': 48, 'temperatureUnit': 'F', 'temperatureTrend': '', 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 0}, 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 0}, 'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 54}, 'windSpeed': '7 mph', 'windDirection': 'W', 'icon': 'https://api.weather.gov/icons/land/night/sct?size=small', 'shortForecast': 'Partly Cloudy', 'detailedForecast': ''}


In [14]:
structured_data = [
    {
        "start_time": p["startTime"],
        "end_time": p["endTime"],
        "temperature": p["temperature"],
        "temperature_unit": p["temperatureUnit"],
        "wind_speed": p["windSpeed"],
        "short_forecast": p["shortForecast"]
    }
    for p in periods
]

print(structured_data[:3])  # show first 3 records

[{'start_time': '2025-11-12T22:00:00-05:00', 'end_time': '2025-11-12T23:00:00-05:00', 'temperature': 48, 'temperature_unit': 'F', 'wind_speed': '7 mph', 'short_forecast': 'Partly Cloudy'}, {'start_time': '2025-11-12T23:00:00-05:00', 'end_time': '2025-11-13T00:00:00-05:00', 'temperature': 47, 'temperature_unit': 'F', 'wind_speed': '8 mph', 'short_forecast': 'Mostly Clear'}, {'start_time': '2025-11-13T00:00:00-05:00', 'end_time': '2025-11-13T01:00:00-05:00', 'temperature': 46, 'temperature_unit': 'F', 'wind_speed': '8 mph', 'short_forecast': 'Mostly Clear'}]


In [18]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="postgres",   # your main database
    user="postgres",       # your username
    password="mysecretpassword"  # blank if using PostgresApp
)
cur = conn.cursor()

In [ ]:
# Insert the structured data into the table

In [19]:
insert_query = """
INSERT INTO weather.hourly_weather
(start_time, end_time, temperature, temperature_unit, wind_speed, short_forecast)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for record in structured_data:
    cur.execute(insert_query, (
        record["start_time"],
        record["end_time"],
        record["temperature"],
        record["temperature_unit"],
        record["wind_speed"],
        record["short_forecast"]
    ))

conn.commit()
print("✅ Data inserted successfully!")

cur.close()
conn.close()


✅ Data inserted successfully!


In [ ]:
# Fetch data via API
# Process & structure JSON
# Store clean data into PostgreSQL
# erify through DBeaver (see in our table)